## Import 1

In [ ]:
# Import 1. 
# Import functions.py into notebook from src folder. 
import sys

# Path relative to your notebook.
sys.path.append("/Users/joel/Desktop/daimil10/capstone_1/DC-Metro-Crime-Data-2007---2017/src")

# Import as usual.
import functions

# Auto reload.
%load_ext autoreload
%autoreload 2

## Import 2

In [ ]:
# Import 2.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
import folium
from folium.plugins import HeatMap

## Read File & Drop First Two Rows

In [ ]:
# Read file in and drop first two rows.
# First two rows seem to make like no sense with the rest of this data set.
# Plan is to drop them.
df = functions.read_file('/Users/joel/Desktop/daimil10/capstone_1/DC-Metro-Crime-Data-2007---2017/data/dc_crime_add_vars.csv')
df = df.drop(['Unnamed: 0', 'X'], axis=1)
df.head()

## Describing the Dataset

In [ ]:
# Describing the Dataset 1.
df.describe()

In [ ]:
# Describing the Dataset 2.
df.info()

In [ ]:
# Describing the Dataset 3.
df.index

In [ ]:
# Describing the Dataset 4.
df.columns

In [ ]:
# Describing the Dataset 5.
missing_v = df.isnull().sum()
missing_v

# To viz - make new dataframe of al missing values.
data = {
    'DISTRICT': [200],
    'PSA': [251],
    'NEIGHBORHOOD_CLUSTER': [4705],
    'BLOCK_GROUP': [1091],
    'CENSUS_TRACT': [1091],
    'VOTING_PRECINCT': [84],
    'START_DATE': [13],
    'END_DATE': [11651],
    'XBLOCK': [0],
    'YBLOCK': [0],
    'date': [0],
    'year': [0],
    'month': [0],
    'day': [0],
    'hour': [0],
    'minute': [0],
    'second': [0]
}

# Show the missing data.
df_bar = pd.DataFrame(data)

# Figsize.
# Use dataframe plot featuer covered in week 4.
plt.figure(figsize=(10, 6))
df_bar.plot(kind='bar', edgecolor='black')

# Labels and title.
plt.xlabel('Col Name')
plt.ylabel('Missing Values')
plt.title('Missing Values in Dataset')

# plt.show().
plt.show()

In [ ]:
# Describing the Dataset 6.
# Went from 342867 to 325340 = 17527 removed which is roughly 5% of the dataset. 
df = functions.drop_rows_with_missing_data(df)
df.info()

In [ ]:
# Describing the Dataset 7.
# Using the function that formats the nubers and spits out df.describe().
functions.describe_dataframe(df)

In [ ]:
# Describing the Dataset 8
# Using the function that spits outdict of col names and dtypes. 
functions.get_column_features(df)

## Slice DF for Only Homicides

In [ ]:
# Can use this is need to or can slice the other df accordingly (really dealer choice).
# Slice df to only include the offense of homicide.

# Convert datetimes.
# https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html 

# Make new start and end hour column.
# Might need this for the corr matrix? 
df_2 = df[df['OFFENSE']=='HOMICIDE']
df_2['START_DATE'] = pd.to_datetime(df_2['START_DATE'])
df_2['END_DATE'] = pd.to_datetime(df_2['END_DATE'])
df_2['start_hour'] = df_2['START_DATE'].dt.hour
df_2['end_hour'] = df_2['START_DATE'].dt.hour
df_2 = df_2.drop('hour', axis=1)
df_2.head()

In [ ]:
# Can use this is need to or can slice the other df accordingly (same statement as above).

# 1100 rows now.
hom_df = df_2

# Slicing df one last time to get only my desired columns.
hom_df = hom_df[['REPORT_DAT',
                'SHIFT',
                'OFFENSE',
                'METHOD',
                'BLOCK',
                'DISTRICT',
                'PSA',
                'WARD',
                'ANC',
                'NEIGHBORHOOD_CLUSTER',
                'BLOCK_GROUP',
                'VOTING_PRECINCT',
                'START_DATE',
                'END_DATE',
                'XBLOCK',
                'YBLOCK',
                'date',
                'year',
                'start_hour',
                'end_hour',
                'crimetype']]

# Show that it worked .. 
hom_df.columns

## Q1: Have homicides increased over this specified decade?

In [ ]:
# Q1: Have homicides increased over this decade?
# Line Chart

# Slice df or I could use the other one above.. 
homicides_df = df[df['OFFENSE'] == 'HOMICIDE']

# Group the data by year and count the number of homicides per year.
# https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.size.html
homicides_by_year = homicides_df.groupby('year').size()

# Make an object out of my PLot class.
plotter = functions.Plot()

# Use line chart method to make my graph! 
plotter.line_chart(homicides_by_year, 'Year', 'Homicides', 'Number of Homicides by Year: 2008 - 2017')

## Q2: Which voting wards had the highest number of homicides?

In [ ]:
# Q2: Which voting wards had the highest number of homicides?
# Bar Chart. 

# Slice df or I could use the other one above.. 
homicides_df = df[df['OFFENSE'] == 'HOMICIDE']

# Get homicides by ward (need value counts and sort values for descending order).
homicides_by_location = homicides_df['WARD'].value_counts().sort_values()

# Already have my plotter object from the line chart. 
# Use bar chart method to make my graph! 
plotter.bar_chart(homicides_by_location, 'Ward', 'Homicides', 'Number of Homicides by Ward')

## Q3: Is there any correlation between the time of the day and the occurrence of homicides?

In [ ]:
# Q3: Is there any correlation between the time of the day and the occurrence of homicides?
# Heat Map.
# This one gets a little funky.

# Extract the hour from the 'REPORT_DAT' column.
# Seems to be more stable then the actaul hour column?
df['Time_of_Day'] = pd.to_datetime(df['REPORT_DAT']).dt.hour

# Create a binary(dummy) column for the category based columns.
# https://codereview.stackexchange.com/questions/251690/create-dummy-variables-in-dataframe-using-for-loop-and-apply-lambda
df['Homicide'] = df['OFFENSE'].apply(lambda x: 1 if 'HOMICIDE' in x else 0)
df['Shift_Evening'] = df['SHIFT'].apply(lambda x: 1 if 'EVENING' in x else 0)
df['Shift_Midnight'] = df['SHIFT'].apply(lambda x: 1 if 'MIDNIGHT' in x else 0)

# Select the columns of interest for corr df.
correlation_df= df[['Time_of_Day', 'Homicide', 'Shift_Evening', 'Shift_Midnight']]
corr_matrix = correlation_df.corr()

# Already have my plotter object from the line chart. 
# Use heatmap method to make my graph! 
plotter.heatmap(corr_matrix, 'Matrix Columns', 'Matrix Column', 'Correlation Heatmap: Homicide & Time of Day')

## Q4: Are homicides more likely to occur with a gun or a knife? 

In [ ]:
# Q4: Are homicides more likely to occur with a gun or a knife? 
# Stacked Plot.

# First name is way to long for graph. 
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html?highlight=replace#pandas.DataFrame.replace
df['OFFENSE'] = df['OFFENSE'].replace('ASSAULT W/DANGEROUS WEAPON', 'ASSAULT W/ WEAPON')

# Select list of 'violent' offenses to put up next to murder to show contrast. 
offenses = ['HOMICIDE', 'ROBBERY', 'ASSAULT W/ WEAPON', 'BURGLARY']

# Filter the DataFrame using isin cause i keep getting value error when I dont do that?
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html 
filtered_df = df[df['OFFENSE'].isin(offenses)]

# Group the data by offense and method, and count.
# .size()is used after grouping the data to count the number of occurrences -
# for each group.
# .unstack() is applied to a hierarchical index (such as the result of a groupby operation) -
# and reshapes the data by "unstacking" one level of the index to create columns. 
# Used the many stacked bar chart examples we have done in the class.
grouped_data = filtered_df.groupby(['OFFENSE', 'METHOD']).size().unstack()

# Calculate the proportions for each method within each offense.
# Wanted to show the precent so just had to times the decimal by 100.
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.div.html was handy even though could of done '/'.
grouped_data_proportions = grouped_data.div(grouped_data.sum(axis=1), axis=0) * 100

# Already have my plotter object from the line chart. 
# Use stacked method to make my graph!
plotter.stacked(grouped_data_proportions, 'Offense', 'Proportion(%)', 'Proportion of Offenses by Method: 2007 - 2017')

## Q5a: Which voting ward has the highest number of homicides over the last decade?

In [ ]:
# Q5a: Which voting ward has the highest number of homicides over the last decade?
# First of two folium heatmaps. 
# Filter the DataFrame to include only homicides.
homicides_df = df[df['OFFENSE'] == 'HOMICIDE']

# Group the data by ward and count the number of homicides in each ward.
WARD_homicide_counts = homicides_df['WARD'].value_counts()

# Find the ward with the highest amount of homicides.
# Used idxmax from previous week's exposure. 
WARD_with_highest_homicides = WARD_homicide_counts.idxmax()

# Filter the DataFrame to include only the data for the ward with the highest amount of homicides.
WARD_homicides_df = homicides_df[homicides_df['WARD'] == WARD_with_highest_homicides]

# Already have my plotter object from the line chart. 
# Use folium_heat method to make my graph!
plotter.folium_heat(WARD_homicides_df, homicides_df, 'WARD', 'WARD 8', 8, 38.8457, -77.0097)

## Q5b: Which police service area had the highest number of homicides over the last decade?

In [ ]:
# Q5b: Which police service area had the highest number of homicides over the last decade?
# Second of two folium heatmaps. 
# Filter the DataFrame to include only homicides.
homicides_df = df[df['OFFENSE'] == 'HOMICIDE']

# Group the data by PSA and count the number of homicides in each PSA.
PSA_homicide_counts = homicides_df['PSA'].value_counts()

# Find the PSA with the highest amount of homicides. 
# Used idxmax from previous week's exposure. 
PSA_with_highest_homicides = PSA_homicide_counts.idxmax()

# Filter the DataFrame to include only the data for the PSA with the highest amount of homicides.
PSA_homicides_df = homicides_df[homicides_df['PSA'] == PSA_with_highest_homicides]

# Already have my plotter object from the line chart. 
# Use folium_heat method to make my graph!
plotter.folium_heat(PSA_homicides_df, homicides_df, 'PSA', 'PSA 604', 604, 38.8851, -76.9158)